# Image classification 

In [1]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [61]:
# Import the necessary modules
import zipfile,os
import shutil
from sklearn.model_selection import train_test_split

# Define the path of the zip file
local_zip = 'rockpaperscissors.zip'

# Open the zip file in read mode
zip_ref = zipfile.ZipFile(local_zip, 'r')

# Extract all the contents of the zip file in /tmp directory
zip_ref.extractall('/tmp')

# Close the ZipFile object
# zip_ref.close()

# Define the base directory where the images are located
base_dir = '/tmp/rockpaperscissors/rps-cv-images'

# Define the directory for the training data
rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors') 

In [62]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Split the data by train/test for each directory
train_rock_dir, val_rock_dir = train_test_split(os.listdir(rock_dir), test_size = 0.4)
train_paper_dir, val_paper_dir = train_test_split(os.listdir(paper_dir), test_size = 0.4)
train_scissors_dir, val_scissors_dir = train_test_split(os.listdir(scissors_dir), test_size = 0.4)

In [63]:
# Function to move files
def move_files(files, src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)  # Ensure the directory exists
    for file in files:
        shutil.move(os.path.join(src_dir, file), os.path.join(dst_dir, file))

# Move the files
move_files(train_rock_dir, rock_dir, os.path.join(train_dir, 'rock'))
move_files(val_rock_dir, rock_dir, os.path.join(validation_dir, 'rock'))

move_files(train_paper_dir, paper_dir, os.path.join(train_dir, 'paper'))
move_files(val_paper_dir, paper_dir, os.path.join(validation_dir, 'paper'))

move_files(train_scissors_dir, scissors_dir, os.path.join(train_dir, 'scissors'))
move_files(val_scissors_dir, scissors_dir, os.path.join(validation_dir, 'scissors'))

In [55]:
rocklen = len(os.listdir('/tmp/rockpaperscissors/rps-cv-images\\rock'))
print(f'Jumlah gambar rock : {rocklen}')

paperlen = len(os.listdir('/tmp/rockpaperscissors/rps-cv-images\\paper'))
print(f'Jumlah gambar paper : {paperlen}')

scissorslen = len(os.listdir('/tmp/rockpaperscissors/rps-cv-images\\paper'))
print(f'Jumlah gambar paper : {scissorslen}')

Jumlah gambar rock : 726
Jumlah gambar paper : 712
Jumlah gambar paper : 712


In [40]:
rock_data = os.listdir(rock_dir)
train_rock, val_rock = train_test_split(rock_data, test_size=0.4)
len(train_rock)

435

In [47]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets for each category
rock_data = os.listdir(rock_dir)
train_rock, val_rock = train_test_split(rock_data, test_size=0.4, random_state=5071)

paper_data = os.listdir(paper_dir)
train_paper, val_paper = train_test_split(paper_data, test_size=0.4, random_state=5071)

scissors_data = os.listdir(scissors_dir)
train_scissors, val_scissors = train_test_split(scissors_data, test_size=0.4, random_state=5071)

In [48]:
print(len(train_rock))
print(len(train_paper))
print(len(train_scissors))

435
427
450


In [49]:
train_data = train_rock + train_paper + train_scissors
val_data = val_rock + val_paper + val_scissors

In [51]:
print(f'Jumlah data train: {len(train_data)}')  
print('      ')
print(f'Jumlah data test: {len(val_data)}')  

Jumlah data train: 1312
      
Jumlah data test: 876


In [84]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # should contain 3 subdirectories, one for each class
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'  # 'categorical' for multi-class labels
)

Found 1814 images belonging to 3 classes.


In [85]:
test_datagen = ImageDataGenerator(
                    rescale=1./255
)


validation_generator = test_datagen.flow_from_directory(
    validation_dir,  # should contain 3 subdirectories, one for each class
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'  # 'categorical' for multi-class labels
)

Found 1378 images belonging to 3 classes.


In [86]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : paper
1 : rock
2 : scissors


In [87]:
labels

{0: 'paper', 1: 'rock', 2: 'scissors'}

In [88]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)), 
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(3, activation='softmax')
])   

In [89]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 17, 17, 128)      

In [90]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [91]:
model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=2
)

Epoch 1/15
91/91 - 39s - loss: 0.8119 - accuracy: 0.6080 - val_loss: 0.2671 - val_accuracy: 0.9136 - 39s/epoch - 426ms/step
Epoch 2/15
91/91 - 33s - loss: 0.2733 - accuracy: 0.9013 - val_loss: 0.2155 - val_accuracy: 0.9238 - 33s/epoch - 359ms/step
Epoch 3/15
91/91 - 36s - loss: 0.1997 - accuracy: 0.9305 - val_loss: 0.1898 - val_accuracy: 0.9311 - 36s/epoch - 393ms/step
Epoch 4/15
91/91 - 32s - loss: 0.1542 - accuracy: 0.9520 - val_loss: 0.1178 - val_accuracy: 0.9601 - 32s/epoch - 347ms/step
Epoch 5/15
91/91 - 29s - loss: 0.1305 - accuracy: 0.9537 - val_loss: 0.2397 - val_accuracy: 0.9231 - 29s/epoch - 319ms/step
Epoch 6/15
91/91 - 29s - loss: 0.1129 - accuracy: 0.9636 - val_loss: 0.0798 - val_accuracy: 0.9710 - 29s/epoch - 319ms/step
Epoch 7/15
91/91 - 31s - loss: 0.0935 - accuracy: 0.9653 - val_loss: 0.0438 - val_accuracy: 0.9920 - 31s/epoch - 342ms/step
Epoch 8/15
91/91 - 36s - loss: 0.0655 - accuracy: 0.9813 - val_loss: 0.0325 - val_accuracy: 0.9920 - 36s/epoch - 400ms/step
Epoch 9/

In [44]:
from PIL import Image

# Open an image file
img = Image.open('../tangan.png')
# Get image size
width, height = img.size

print(f"The image size is {width} pixels wide and {height} pixels high.")

The image size is 300 pixels wide and 200 pixels high.
